In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use("ggplot")



In [ ]:
df = pd.read_csv("Amazon Sale Report.csv")
df.columns = [c.strip() for c in df.columns]

df.head()


In [ ]:
df["Date_parsed"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")

# Detect amount column
amount_col = None
for c in ["Amount","amount","Total Amount","Price","Order Value"]:
    if c in df.columns:
        amount_col = c
        break

df["Amount_numeric"] = pd.to_numeric(df[amount_col], errors="coerce")

# Status flags
status = df["Status"].astype(str).str.lower()
df["is_cancelled"] = status.str.contains("cancel", na=False)
df["is_return"] = status.str.contains("return", na=False)
df["is_completed"] = ~(df["is_cancelled"] | df["is_return"])

df.head()


In [ ]:
total_orders = df["Order ID"].nunique()
completed_orders = df.loc[df["is_completed"], "Order ID"].nunique()
total_revenue = df.loc[df["is_completed"], "Amount_numeric"].sum()
aov = total_revenue / completed_orders
cancel_return_rate = (df["is_cancelled"] | df["is_return"]).sum() / total_orders

kpi_data = {
    "Total Orders": total_orders,
    "Completed Orders": completed_orders,
    "Total Revenue": total_revenue,
    "AOV": aov,
    "Cancel+Return Rate": cancel_return_rate
}

pd.DataFrame([kpi_data])


In [ ]:
monthly = (
    df[df["is_completed"]]
    .groupby(pd.Grouper(key="Date_parsed", freq="M"))["Amount_numeric"]
    .sum()
    .reset_index()
)

plt.figure(figsize=(10,4))
plt.plot(monthly["Date_parsed"], monthly["Amount_numeric"], marker="o")
plt.title("Monthly Revenue")
plt.xlabel("Month")
plt.ylabel("Revenue (INR)")
plt.show()


In [ ]:
category_rev = (
    df[df["is_completed"]]
    .groupby("Category")["Amount_numeric"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure(figsize=(10,4))
sns.barplot(x=category_rev.index, y=category_rev.values)
plt.xticks(rotation=30)
plt.title("Top 10 Categories")
plt.show()


In [ ]:
ful_col = None
for c in ["Fulfilment","Fulfillment"]:
    if c in df.columns:
        ful_col = c
        break

ful = df.groupby(ful_col).agg(
    Orders=("Order ID","nunique"),
    Revenue=("Amount_numeric", lambda s: df.loc[s.index & df.index,"Amount_numeric"][df.loc[s.index & df.index,"is_completed"]].sum())
)

ful


In [ ]:
out = Path("outputs/tables")
out.mkdir(parents=True, exist_ok=True)

# KPIs
pd.DataFrame([kpi_data]).to_csv(out/"kpis.csv", index=False)

# Monthly revenue
monthly.to_csv(out/"monthly_revenue.csv", index=False)

# Top categories
category_rev.reset_index().to_csv(out/"top_categories.csv", index=False)

"Export Complete!"
